In [ ]:
!pip install langchain-community langchain-huggingface chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
!pip install transformers=='4.49.0' peft=='0.10.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 118.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
  Attempting uninstall: peft
    Found existing installation: peft 0.18.0
    Uninstalling peft-0.18.0:
      Successfully uninstalled peft-0.18.0


In [1]:
import os
import pandas as pd
import torch
import google.generativeai as genai
import gradio as gr
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


c:\Users\Вера\Desktop\projects\genai\course project\photo-recipe-generator\venv\lib\site-packages\google\api_core\_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.12) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
c:\Users\Вера\Desktop\projects\genai\course project\photo-recipe-generator\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Вера\AppData\Local\Temp\ipykernel_26644\3978419103.py:4: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please s

In [ ]:
GEMINI_API_KEY = "YOUR_GEMINI_TOKEN"
DB_PATH = "recipe_db"
CSV_PATH = "data\all_recepies_inter.csv" # Можно использовать любую другую таблицу с рецептами, поменять структуру под себя

# Настройка Gemini
genai.configure(api_key=GEMINI_API_KEY)
llm_model = genai.GenerativeModel('gemini-2.5-flash')

In [3]:
# Модель эмбеддингов (общая для базы и для поиска)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Загрузка Florence-2 (Vision)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Загрузка Florence-2 на {device}...")
vlm_model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True).to(device).eval()
vlm_processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)

Загрузка Florence-2 на cpu...


In [4]:
def initialize_vector_db():
    if os.path.exists(DB_PATH):
        print("Векторная база уже существует. Загружаем...")
        return Chroma(persist_directory=DB_PATH, embedding_function=embedding_model)

    print("Создание векторной базы из CSV...")
    df = pd.read_csv(CSV_PATH, sep='\t')

    docs = []
    for _, row in df.iterrows():
        # То, по чему ищем (контент для эмбеддинга)
        search_content = f"Название: {row['name']} Состав: {row['composition']}"
        # То, что храним (метаданные)
        metadata = {
            "name": str(row['name']),
            "composition": str(row['composition']),
            "instructions": str(row['Инструкции'])
        }
        docs.append(Document(page_content=search_content, metadata=metadata))

    # Создание и сохранение базы
    db = Chroma.from_documents(docs, embedding_model, persist_directory=DB_PATH)
    print("База успешно создана!")
    return db

# Инициализируем базу один раз при запуске
vector_db = initialize_vector_db()

Векторная база уже существует. Загружаем...


C:\Users\Вера\AppData\Local\Temp\ipykernel_26644\4035970699.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(persist_directory=DB_PATH, embedding_function=embedding_model)


In [5]:
def recipe_pipeline(input_image):
    if input_image is None:
        return "Пожалуйста, загрузите изображение."

    # ШАГ А: Florence-2 (Распознавание продуктов)
    inputs = vlm_processor(text="<DETAILED_CAPTION>", images=input_image, return_tensors="pt").to(device)
    generated_ids = vlm_model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=512,
        num_beams=3
    )
    generated_text = vlm_processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed = vlm_processor.post_process_generation(generated_text, task="<DETAILED_CAPTION>", image_size=(input_image.width, input_image.height))
    detected_ingredients = parsed["<DETAILED_CAPTION>"]

    # ШАГ Б: Поиск в векторной базе (RAG)
    # Текст от Florence-2 превращается в вектор и сравнивается с векторами в ChromaDB
    found_docs = vector_db.similarity_search(detected_ingredients, k=1)
    if not found_docs:
        return "В базе не найдено подходящего рецепта."

    source = found_docs[0].metadata

    # ШАГ В: Генерация через Gemini
    prompt = f"""
    Ты - профессиональный шеф-повар. К тебе пришел клиент с фото продуктов.

    ЧТО ВИДИТ ИИ НА ФОТО: {detected_ingredients}

    НАЙДЕННЫЙ РЕЦЕПТ В БАЗЕ (CSV):
    Название: {source['name']}
    Оригинальный состав: {source['composition']}
    Оригинальная инструкция: {source['instructions']}

    ЗАДАЧА:
    Адаптируй этот рецепт. Если на фото есть лишние ингредиенты - предложи как их добавить.
    Если чего-то из базы нет на фото - предложи замену или скажи, что это можно пропустить.
    Выдай красивый пошаговый рецепт на русском языке.
    """

    response = llm_model.generate_content(prompt)
    return response.text

In [9]:
with gr.Blocks(theme=gr.themes.Monochrome()) as interface:
    gr.Markdown("# ИИ-Шеф: Генератор рецептов по фото")
    gr.Markdown("Загрузите фото, Florence-2 распознает продукты, RAG найдет рецепт в CSV, а Gemini его адаптирует.")

    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="pil", label="Загрузите фото ингредиентов")
            run_btn = gr.Button("Создать рецепт", variant="primary")

        with gr.Column():
            recipe_output = gr.Markdown(label="Адаптированный рецепт")

    run_btn.click(fn=recipe_pipeline, inputs=img_input, outputs=recipe_output)
    interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://61fa35da9fde49f05f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
